In [7]:
pip install dice-ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import numpy as np
import timeit
import random

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.model_selection import train_test_split

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We use the "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). For demonstration purposes, we transform the data as described in dice_ml.utils.helpers module.

In [10]:
dataset = helpers.load_adult_income_dataset()

In [11]:
dataset

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0
...,...,...,...,...,...,...,...,...,...
26043,28,Private,HS-grad,Married,White-Collar,White,Male,40,0
26044,18,Private,School,Single,Blue-Collar,White,Male,55,0
26045,22,Private,Some-college,Single,White-Collar,White,Female,40,0
26046,42,Self-Employed,Bachelors,Divorced,White-Collar,Other,Male,30,0


DiCE supports sklearn, tensorflow and pytorch models.

The variable backend below indicates the implementation type of DiCE we want to use. Four backends are supported: sklearn, TensorFlow 1.x with backend='TF1', Tensorflow 2.x with backend='TF2', and PyTorch with backend='PYT'.

Below we show use a trained classification model using sklearn.

In [12]:
dataset = helpers.load_adult_income_dataset()
target = dataset["income"] # outcome variable
train_dataset, test_dataset, _, _ = train_test_split(dataset,
                                                     target,
                                                     test_size=0.2,
                                                     random_state=0,
                                                     stratify=target)
# Dataset for training an ML model
d = dice_ml.Data(dataframe=train_dataset,
                 continuous_features=['age', 'hours_per_week'],
                 outcome_name='income')

# Pre-trained ML model
m = dice_ml.Model(model_path=dice_ml.utils.helpers.get_adult_income_modelpath(),
                  backend='TF2', func="ohe-min-max")
# DiCE explanation instance
exp = dice_ml.Dice(d,m)


With DiCE, generating explanations is a simple three-step process: set up a dataset, train a model, and then invoke DiCE to generate counterfactual examples for any input. DiCE can also work with pre-trained models, with or without their original training data.

For any given input, we can now generate counterfactual explanations. For example, the following input leads to class 0 (low income) and we would like to know what minimal changes would lead to a prediction of 1 (high income).

In [13]:
query_instance = test_dataset.drop(columns="income")[0:1]
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=10, desired_class="opposite")
# Visualize counterfactual explanation
dice_exp.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:01<00:00,  1.33s/it]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Prof-school,-,-,-,-,63.0,1
1,75.0,-,Masters,-,-,-,-,-,1
2,79.0,-,Doctorate,-,-,-,-,-,1
3,71.0,-,Assoc,-,-,-,-,-,1
4,-,-,Masters,-,-,-,-,-,1
5,-,-,Assoc,-,-,-,-,64.0,1
6,57.0,-,-,-,Service,-,-,-,1
7,-,-,-,-,-,-,-,76.0,1
8,-,-,-,-,-,-,-,68.0,1
9,-,-,Some-college,-,-,-,-,80.0,1


In [14]:
query_instance = test_dataset.drop(columns="income")[0:1]
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=10, desired_class="opposite",proximity_weight=1.5)
# Visualize counterfactual explanation
dice_exp.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:01<00:00,  1.33s/it]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,Other/Unknown,Prof-school,-,-,-,-,-,1
1,-,-,Doctorate,-,-,-,-,14.0,1
2,68.0,-,-,-,Professional,-,-,-,1
3,-,-,-,-,-,-,-,81.0,1
4,-,Self-Employed,-,-,Professional,-,-,-,1
5,-,Self-Employed,-,-,-,-,-,79.0,1
6,-,-,Bachelors,-,-,-,-,58.0,1
7,-,-,-,-,-,-,-,84.0,1
8,-,Self-Employed,-,-,Sales,-,-,-,1
9,-,-,Assoc,-,-,-,-,39,1
